## Critical Sections: The Synchronization Problem

We are now going to take a deeper look inside synchronization of threads/processes and the principles needed to share data safely. There are two typical goals:
* Contention: 
  * How to resolve the conflicts that result from multiple processes trying to access shared resources?
* Cooperation:
  * An action by one process may enable another action by another process
  * In such cases, processes should coordinate their actions
  
Maybe do the parable of buying milk to motivate why it is hard....... Or find a better example.

Synchronization issues are the major bug in parallel programs
* Deadlock: when a program cannot progress because all of it's threads/processes are waiting on resources.
* Incorrect results: from uncontrolled sharing (race conditions) on data.

The constructs/algorithms underlying critical sections (locks, atomic variables) are complex.
Understanding them will help you use them well.

  
### Mutual Exclusion

Mutual exclusion between two processes is the fundamental problemn in 


